In [1]:
import pandas as pd
import numpy as np
from openai import AzureOpenAI

In [2]:
news=pd.read_csv(r"./新闻快讯_新浪.csv")

In [3]:
news

,datetime,content,title
0,2018-10-11 23:59:10,现货白银扩大涨幅至2%，现报14.56美元/盎司。,NaN
1,2018-10-11 23:53:08,纳指再度转涨。,NaN
2,2018-10-11 23:49:17,哥伦比亚广播下跌2.2%。,NaN
3,2018-10-11 23:45:47,现货黄金涨破1220美元/盎司关口，日内涨2.16%；现货白银现报14.54美元/盎司，日内...,NaN
4,2018-10-11 23:41:29,美国总统特朗普：美联储引发市场调整，美联储正在犯错。不会解雇美联储主席鲍威尔。,NaN
...,...,...,...
1376205,2024-03-19 10:40:29,黑色系期货短线走高，铁矿石主力合约涨超5%，焦煤主力合约涨超3%，热卷、螺纹钢、焦炭主力合约...,NaN
1376206,2024-03-19 10:39:50,焦煤主力合约涨超3%，现报1623.5元/吨；焦炭主力合约日内涨幅达2.00%，现报2169...,NaN
1376207,2024-03-19 10:38:44,【北向资金净卖出超40亿元】 截至目前北向资金净卖出超41亿元，其中沪股通净卖出26.15亿...,NaN
1376208,2024-03-19 10:38:30,【北京市市长殷勇：扩大外调绿电规模 力争绿电占比达到25%】国务院新闻办公室上午举行“推动高...,NaN


In [ ]:
news[news[]]

In [6]:
sample_news=news.iloc[:100,:]
sample_news

,datetime,content,title
0,2018-10-11 23:59:10,现货白银扩大涨幅至2%，现报14.56美元/盎司。,NaN
1,2018-10-11 23:53:08,纳指再度转涨。,NaN
2,2018-10-11 23:49:17,哥伦比亚广播下跌2.2%。,NaN
3,2018-10-11 23:45:47,现货黄金涨破1220美元/盎司关口，日内涨2.16%；现货白银现报14.54美元/盎司，日内...,NaN
4,2018-10-11 23:41:29,美国总统特朗普：美联储引发市场调整，美联储正在犯错。不会解雇美联储主席鲍威尔。,NaN
...,...,...,...
95,2018-10-11 21:02:02,FTNT加入标普500指数。,NaN
96,2018-10-11 21:01:26,在岸人民币兑美元逼近6.89关口，较日内低点涨逾400点。,NaN
97,2018-10-11 21:01:02,摩根大通将HES目标价上调至68美元，此前为64美元。,NaN
98,2018-10-11 21:00:56,【夜盘开盘，原油期货领跌】原油期货跌1.5%，沪镍、燃油跌1%。橡胶、郑棉、沥青、豆二等开盘...,NaN


## GPT Training

In [58]:
client=AzureOpenAI(
    api_key="42f6fa30e87f44bf895f7f5f4d4cd28c",
    api_version="2023-05-15",
    azure_endpoint="https://hkust.azure-api.net"
)

In [61]:
#test of first agent
sample_size=20
sample=news.iloc[:sample_size,:]
news_features = []
for i in range(sample_size):#len(news
    content=news.iloc[i,1]
    response=client.chat.completions.create(
    model="gpt-35-turbo",
    messages=[
        {"role": "system", "content": "你是一个中立的金融学者，可以判断财经新闻的种类和信息。现在我将输入新闻标题，请你判断该条新闻是否包括两家或更多家公司的明确的产业链信息，不含有则输出0，含有则输出1。请不要输出多余的信息和标点符号，只输出一个数字。"},
        {"role": "user", "content": content},
    ],
    )
    features = response.choices[0].message.content
    news_features.append(features)
columns = ['whether_include_supply_chain']
news_features = pd.DataFrame(news_features, columns=columns)
new_sample=pd.concat([sample, news_features], axis=1)
new_sample

,datetime,content,title,whether_include_supply_chain
0,2018-10-11 23:59:10,现货白银扩大涨幅至2%，现报14.56美元/盎司。,NaN,0
1,2018-10-11 23:53:08,纳指再度转涨。,NaN,0
2,2018-10-11 23:49:17,哥伦比亚广播下跌2.2%。,NaN,0
3,2018-10-11 23:45:47,现货黄金涨破1220美元/盎司关口，日内涨2.16%；现货白银现报14.54美元/盎司，日内...,NaN,0
4,2018-10-11 23:41:29,美国总统特朗普：美联储引发市场调整，美联储正在犯错。不会解雇美联储主席鲍威尔。,NaN,0
5,2018-10-11 23:40:49,日本财政部高级官员：如果市场走势仍然非常不稳定，就必须考虑某种回应措施。,NaN,0
6,2018-10-11 23:38:56,在岸人民币兑美元（CNY）北京时间23:30收报6.8888元，较周三夜盘收盘涨352点；全...,NaN,0
7,2018-10-11 23:32:29,焦炭夜盘收涨1.12%，焦煤收涨0.22%，动力煤收涨0.03%；铁矿石收涨0.39%。橡胶...,NaN,0
8,2018-10-11 23:31:13,富时泛欧绩优300指数初步收跌1.98%。,NaN,0
9,2018-10-11 23:29:04,洛克希德马丁下跌1.6%。,NaN,0


In [52]:
#second agent start
def extract_news_features(response):
    features = response.split(';')
    feature_num=12
    if(len(features)==feature_num):
        try:
            news_type = features[0].split(':')[1].strip()
            industry = features[1].split(':')[1].strip()
            main_company = features[2].split(':')[1].strip()
            related_companies = features[3].split(':')[1].strip()
            business_relation = features[4].split(':')[1].strip()
            region_impact = features[5].split(':')[1].strip()
            impact_on_china_market = features[6].split(':')[1].strip()
            market_impact_index = (features[7].split(':')[1].strip())
            investor_sentiment = features[8].split(':')[1].strip()
            investor_sentiment_index = (features[9].split(':')[1].strip())
            prediction_time_range = features[10].split(':')[1].strip()
            prediction_heat = (features[11].split(':')[1].strip())
            return [news_type, industry, main_company, related_companies, business_relation, 
                region_impact, impact_on_china_market, market_impact_index, investor_sentiment,
                investor_sentiment_index, prediction_time_range, prediction_heat]
        except IndexError as e:
            nan_values = np.full(feature_num, np.nan).tolist()
            return nan_values

        
    else:
        nan_values = np.full(feature_num, np.nan).tolist()
        return nan_values

In [18]:
pd.set_option('display.max_columns', None)

In [53]:
#test of second agent
sample_size=50
sample=news.iloc[:sample_size,:]
news_features = []
for i in range(sample_size):#len(news
    content=news.iloc[i,1]
    response=client.chat.completions.create(
    model="gpt-35-turbo",
    messages=[
        {"role": "system", "content": "你是一个中立的金融学者，可以判断财经新闻的种类和信息。现在我将输入新闻标题，请你输出新闻特征:新闻分类、行业、主体公司、相关公司、商业关系、地区影响、主体公司是否与中国有直接关系或为A股上市公司、新闻对股市的影响力指数、投资者对新闻的情绪、相关股票投资者情绪强烈指数、新闻对股市的预测时效性(天)、新闻预测热度，每个特征用分号隔开。其中宏观是指宏观的信息，一般与整体市场有关，宏观的行业有政府政策、财政预算、税收政策、商品价格、产业概况、股票指数、汇率、利率、期货价格、贵金属、GDP、通胀、就业等。微观新闻只涉及到一个或一些公司，微观新闻的行业就是公司行业。商业关系的取值范围是-1到1，-1代表主体公司和相关公司是完全竞争，1代表主体公司和相关公司是完全合作或字母公司关系。相关股票投资者情绪强烈指数的取值范围是0到1，数值越大表示相关股票投资者的情绪越激烈。新闻对股市的影响力指数的取值范围是0到1，代表对股票波动的影响力。新闻热度的取值范围是0到1，取值越高代表热度越高。请注意最后不要输出句号。比如输入某条新闻，输出格式是新闻种类:宏观;行业:税收政策;主体公司:无;相关公司:无;商业关系:无;地区影响:有;主体公司是否与中国有直接关系或为A股上市公司:是;新闻对股市的影响力指数:0.2;投资者对新闻的情绪:好;相关股票投资者情绪强烈指数:0.3;新闻对股市的预测时效性（天）:2;新闻预测热度:0.8"},
        {"role": "user", "content": content},
    ],
    )
    features = extract_news_features(response.choices[0].message.content)
    news_features.append(features)
columns = ['news_type', 'industry', 'main_company', 'related_companies', 'business_relation', 
          'region_impact', 'impact_on_china_market', 'market_impact_index', 'investor_sentiment',
          'investor_sentiment_index', 'prediction_timeframe', 'prediction_heat']
news_features = pd.DataFrame(news_features, columns=columns)
new_sample=pd.concat([sample, news_features], axis=1)
new_sample

,datetime,content,title,news_type,industry,main_company,related_companies,business_relation,region_impact,impact_on_china_market,market_impact_index,investor_sentiment,investor_sentiment_index,prediction_timeframe,prediction_heat
0,2018-10-11 23:59:10,现货白银扩大涨幅至2%，现报14.56美元/盎司。,NaN,宏观,贵金属,无,无,无,全球,否,0.1,好,0,1,0.5
1,2018-10-11 23:53:08,纳指再度转涨。,NaN,宏观,股票指数,无,无,无,无,否,0.5,好,0.4,1,0.6
2,2018-10-11 23:49:17,哥伦比亚广播下跌2.2%。,NaN,微观,媒体,哥伦比亚广播公司,无,-1,有,否,0.2,差,0.5,1,0.6
3,2018-10-11 23:45:47,现货黄金涨破1220美元/盎司关口，日内涨2.16%；现货白银现报14.54美元/盎司，日内...,NaN,宏观,贵金属,无,无,无,全球,否,0.1,好,0.2,1,0.6
4,2018-10-11 23:41:29,美国总统特朗普：美联储引发市场调整，美联储正在犯错。不会解雇美联储主席鲍威尔。,NaN,宏观,股票指数,无,美联储,-0.8,无,否,0.4,负面,0.5,1,0.7
5,2018-10-11 23:40:49,日本财政部高级官员：如果市场走势仍然非常不稳定，就必须考虑某种回应措施。,NaN,宏观,财政预算,日本财政部,无,无,有,否,0.3,不确定,0.2,3,0.5
6,2018-10-11 23:38:56,在岸人民币兑美元（CNY）北京时间23:30收报6.8888元，较周三夜盘收盘涨352点；全...,NaN,宏观,汇率,无,无,无,全球,否,0.1,好,0,1,0.5
7,2018-10-11 23:32:29,焦炭夜盘收涨1.12%，焦煤收涨0.22%，动力煤收涨0.03%；铁矿石收涨0.39%。橡胶...,NaN,宏观,商品价格,无,无,无,有,否,0.1,中性,0,1,0.5
8,2018-10-11 23:31:13,富时泛欧绩优300指数初步收跌1.98%。,NaN,宏观,股票指数,无,无,无,全球,否,0.4,差,0.6,1,0.7
9,2018-10-11 23:29:04,洛克希德马丁下跌1.6%。,NaN,微观,航空国防,洛克希德马丁,无,无,无,否,0.1,差,0.2,1,0.5


In [54]:
new_sample.to_csv(r"./new_sample.csv",index=False,encoding='utf_8_sig')